In [2]:
import os
import numpy as np
#import pandas as pd
import matplotlib.pyplot as plt
from random import shuffle
#import pickle, datetime
import math

import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential, load_model
from keras.layers import Input, Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, Convolution2D, MaxPooling2D
from keras.layers.convolutional import ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from keras import optimizers
from keras.preprocessing import sequence
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.callbacks import ModelCheckpoint

import  PIL.Image

In [3]:
root_path = '/Users/jdu12/Desktop/research/'
model_dir = root_path + 'saved_models/GoogleNet_rect/'
train_dir = root_path + 'train/'
val_dir = root_path + 'val/'

In [4]:
# set up input generator
N_CATEGORY = 36
BATCH_SIZE = 32

train_datagen = ImageDataGenerator(horizontal_flip=False)
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=BATCH_SIZE,
    class_mode='categorical')

val_datagen = ImageDataGenerator(horizontal_flip=False)
val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=BATCH_SIZE,
    class_mode='categorical')

Found 4229 images belonging to 36 classes.
Found 846 images belonging to 36 classes.


In [6]:
from __future__ import print_function
from __future__ import absolute_import

import warnings
import numpy as np

from keras.models import Model
from keras import layers
from keras.layers import Input
from keras.layers import Conv2D
from keras.layers import Activation
from keras.layers import BatchNormalization
from keras.layers import MaxPooling2D
from keras.layers import AveragePooling2D
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers import GlobalAveragePooling2D
from keras.layers import GlobalMaxPooling2D
from keras.engine.topology import get_source_inputs
from keras.utils.layer_utils import convert_all_kernels_in_model
from keras.utils.data_utils import get_file
from keras import backend as K
from keras.applications.imagenet_utils import decode_predictions
from keras.applications.imagenet_utils import _obtain_input_shape
from keras.preprocessing import image

WEIGHTS_PATH = 'http://redcatlabs.com/downloads/inception_v1_weights_tf_dim_ordering_tf_kernels.h5'
WEIGHTS_PATH_NO_TOP = 'http://redcatlabs.com/downloads/inception_v1_weights_tf_dim_ordering_tf_kernels_notop.h5'

# conv2d_bn is similar to (but updated from) inception_v3 version
def conv2d_bn(x,
              filters,
              num_row,
              num_col,
              padding='same',
              strides=(1, 1),
              normalizer=True,
              activation='relu',
              name=None):
    """Utility function to apply conv + BN.
    Arguments:
        x: input tensor.
        filters: filters in `Conv2D`.
        num_row: height of the convolution kernel.
        num_col: width of the convolution kernel.
        padding: padding mode in `Conv2D`.
        strides: strides in `Conv2D`.
        name: name of the ops; will become `name + '_conv'`
            for the convolution, `name + '_bn'` for the
            batch norm layer and `name + '_act'` for the
            activation layer.
    Returns:
        Output tensor after applying `Conv2D` and `BatchNormalization`.
    """
    if name is not None:
        conv_name = name + '_conv'
        bn_name = name + '_bn'
        act_name = name + '_act'
    else:
        conv_name = None
        bn_name = None
        act_name = None
    if K.image_data_format() == 'channels_first':
        bn_axis = 1
    else:
        bn_axis = 3
    x = Conv2D(
            filters, (num_row, num_col),
            strides=strides, padding=padding,
            use_bias=False, name=conv_name)(x)
    if normalizer:
        x = BatchNormalization(axis=bn_axis, scale=False, name=bn_name)(x)
    if activation:
        x = Activation(activation, name=act_name)(x)
    return x
    
# Convenience function for 'standard' Inception concatenated blocks
def concatenated_block(x, specs, channel_axis, name):
    (br0, br1, br2, br3) = specs   # ((64,), (96,128), (16,32), (32,))
    
    branch_0 = conv2d_bn(x, br0[0], 1, 1, name=name+"_Branch_0_a_1x1")

    branch_1 = conv2d_bn(x, br1[0], 1, 1, name=name+"_Branch_1_a_1x1")
    branch_1 = conv2d_bn(branch_1, br1[1], 3, 3, name=name+"_Branch_1_b_3x3")

    branch_2 = conv2d_bn(x, br2[0], 1, 1, name=name+"_Branch_2_a_1x1")
    branch_2 = conv2d_bn(branch_2, br2[1], 5, 5, name=name+"_Branch_2_b_5x5")

    branch_3 = MaxPooling2D( (3, 3), strides=(1, 1), padding='same', name=name+"_Branch_3_a_max")(x)  
    branch_3 = conv2d_bn(branch_3, br3[0], 1, 1, name=name+"_Branch_3_b_1x1")

    x = layers.concatenate(
        [branch_0, branch_1, branch_2, branch_3],
        axis=channel_axis,
        name=name+"_Concatenated")
    return x


def InceptionV1(include_top=True,
                weights='imagenet',
                input_tensor=None,
                input_shape=None,
                pooling=None,
                classes=1001):
    """Instantiates the Inception v1 architecture.
    This architecture is defined in:
        Going deeper with convolutions
        Christian Szegedy, Wei Liu, Yangqing Jia, Pierre Sermanet, Scott Reed,
        Dragomir Anguelov, Dumitru Erhan, Vincent Vanhoucke, Andrew Rabinovich.
        http://arxiv.org/abs/1409.4842v1
    
    Optionally loads weights pre-trained
    on ImageNet. Note that when using TensorFlow,
    for best performance you should set
    `image_data_format="channels_last"` in your Keras config
    at ~/.keras/keras.json.
    The model and the weights are compatible with both
    TensorFlow and Theano. The data format
    convention used by the model is the one
    specified in your Keras config file.
    Note that the default input image size for this model is 224x224.
    Arguments:
        include_top: whether to include the fully-connected
            layer at the top of the network.
        weights: one of `None` (random initialization)
            or "imagenet" (pre-training on ImageNet).
        input_tensor: optional Keras tensor (i.e. output of `layers.Input()`)
            to use as image input for the model.
        input_shape: optional shape tuple, only to be specified
            if `include_top` is False (otherwise the input shape
            has to be `(224, 224, 3)` (with `channels_last` data format)
            or `(3, 224, 224)` (with `channels_first` data format).
            It should have exactly 3 inputs channels,
            and width and height should be no smaller than 139.
            E.g. `(150, 150, 3)` would be one valid value.
        pooling: Optional pooling mode for feature extraction
            when `include_top` is `False`.
            - `None` means that the output of the model will be
                the 4D tensor output of the
                last convolutional layer.
            - `avg` means that global average pooling
                will be applied to the output of the
                last convolutional layer, and thus
                the output of the model will be a 2D tensor.
            - `max` means that global max pooling will
                be applied.
        classes: optional number of classes to classify images
            into, only to be specified if `include_top` is True, and
            if no `weights` argument is specified.
    Returns:
        A Keras model instance.
    Raises:
        ValueError: in case of invalid argument for `weights`,
            or invalid input shape.
    """
    if weights not in {'imagenet', None}:
        raise ValueError('The `weights` argument should be either '
                         '`None` (random initialization) or `imagenet` '
                         '(pre-training on ImageNet).')

    if weights == 'imagenet' and include_top and classes != 1001:
        raise ValueError('If using `weights` as imagenet with `include_top`'
                         ' as true, `classes` should be 1001')

    # Determine proper input shape
    input_shape = _obtain_input_shape(
        input_shape,
        #default_size=299,
        default_size=224,
        min_size=139,
        data_format=K.image_data_format(),
        require_flatten=include_top)

    if input_tensor is None:
        img_input = Input(shape=input_shape)
    else:
        img_input = Input(tensor=input_tensor, shape=input_shape)

    if K.image_data_format() == 'channels_first':
        channel_axis = 1
    else:
        channel_axis = 3

    # 'Sequential bit at start'
    x = img_input
    x = conv2d_bn(x,  64, 7, 7, strides=(2, 2), padding='same',  name='Conv2d_1a_7x7')  
    
    x = MaxPooling2D((3, 3), strides=(2, 2), padding='same', name='MaxPool_2a_3x3')(x)  
    
    x = conv2d_bn(x,  64, 1, 1, strides=(1, 1), padding='same', name='Conv2d_2b_1x1')  
    x = conv2d_bn(x, 192, 3, 3, strides=(1, 1), padding='same', name='Conv2d_2c_3x3')  
    
    x = MaxPooling2D((3, 3), strides=(2, 2), padding='same', name='MaxPool_3a_3x3')(x)  
    
    # Now the '3' level inception units
    x = concatenated_block(x, (( 64,), ( 96,128), (16, 32), ( 32,)), channel_axis, 'Mixed_3b')
    x = concatenated_block(x, ((128,), (128,192), (32, 96), ( 64,)), channel_axis, 'Mixed_3c')

    x = MaxPooling2D((3, 3), strides=(2, 2), padding='same', name='MaxPool_4a_3x3')(x)  

    # Now the '4' level inception units
    x = concatenated_block(x, ((192,), ( 96,208), (16, 48), ( 64,)), channel_axis, 'Mixed_4b')
    x = concatenated_block(x, ((160,), (112,224), (24, 64), ( 64,)), channel_axis, 'Mixed_4c')
    x = concatenated_block(x, ((128,), (128,256), (24, 64), ( 64,)), channel_axis, 'Mixed_4d')
    x = concatenated_block(x, ((112,), (144,288), (32, 64), ( 64,)), channel_axis, 'Mixed_4e')
    x = concatenated_block(x, ((256,), (160,320), (32,128), (128,)), channel_axis, 'Mixed_4f')

    x = MaxPooling2D((2, 2), strides=(2, 2), padding='same', name='MaxPool_5a_2x2')(x)  

    # Now the '5' level inception units
    x = concatenated_block(x, ((256,), (160,320), (32,128), (128,)), channel_axis, 'Mixed_5b')
    x = concatenated_block(x, ((384,), (192,384), (48,128), (128,)), channel_axis, 'Mixed_5c')
    

    if include_top:
        # Classification block
        
        # 'AvgPool_0a_7x7'
        x = AveragePooling2D((7, 7), strides=(1, 1), padding='valid')(x)  
        
        # 'Dropout_0b'
        x = Dropout(0.2)(x)  # slim has keep_prob (@0.8), keras uses drop_fraction
        
        #logits = conv2d_bn(x,  classes, 1, 1, strides=(1, 1), padding='valid', name='Logits',
        #                   normalizer=False, activation=None, )  
        
        # Write out the logits explictly, since it is pretty different
        x = Conv2D(classes, (1, 1), strides=(1,1), padding='valid', use_bias=True, name='Logits')(x)
        
        x = Flatten(name='Logits_flat')(x)
        #x = x[:, 1:]  # ??Shift up so that first class ('blank background') vanishes
        # Would be more efficient to strip off position[0] from the weights+bias terms directly in 'Logits'
        
        x = Activation('softmax', name='Predictions')(x)
    else:
        if pooling == 'avg':
            x = GlobalAveragePooling2D(name='global_pooling')(x)
        elif pooling == 'max':
            x = GlobalMaxPooling2D(    name='global_pooling')(x)

    # Ensure that the model takes into account
    # any potential predecessors of `input_tensor`.
    if input_tensor is not None:
        inputs = get_source_inputs(input_tensor)
    else:
        inputs = img_input
        
    # Finally : Create model
    model = Model(inputs, x, name='inception_v1')
    
    # LOAD model weights
    if weights == 'imagenet':
        if K.image_data_format() == 'channels_first':
            if K.backend() == 'tensorflow':
                warnings.warn('You are using the TensorFlow backend, yet you '
                              'are using the Theano '
                              'image data format convention '
                              '(`image_data_format="channels_first"`). '
                              'For best performance, set '
                              '`image_data_format="channels_last"` in '
                              'your Keras config '
                              'at ~/.keras/keras.json.')
        if include_top:
            weights_path = get_file(
                'inception_v1_weights_tf_dim_ordering_tf_kernels.h5',
                WEIGHTS_PATH,
                cache_subdir='models',
                md5_hash='723bf2f662a5c07db50d28c8d35b626d')
        else:
            weights_path = get_file(
                'inception_v1_weights_tf_dim_ordering_tf_kernels_notop.h5',
                WEIGHTS_PATH_NO_TOP,
                cache_subdir='models',
                md5_hash='6fa8ecdc5f6c402a59909437f0f5c975')
        model.load_weights(weights_path)
        if K.backend() == 'theano':
            convert_all_kernels_in_model(model)    
    
    return model



def preprocess_input(x):
    x /= 255.
    x -= 0.5
    x *= 2.
    return x

'''
if __name__ == '__main__':
    model = InceptionV1(include_top=True, weights='imagenet')

    img_path = 'elephant.jpg'
    img = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)

    x = preprocess_input(x)

    preds = model.predict(x)
    
    # Extra shift to remove 'background' as entry0
    preds_1000 = preds[:, 1:]
    
    print('Predicted:', decode_predictions(preds_1000))
'''

"\nif __name__ == '__main__':\n    model = InceptionV1(include_top=True, weights='imagenet')\n\n    img_path = 'elephant.jpg'\n    img = image.load_img(img_path, target_size=(224, 224))\n    x = image.img_to_array(img)\n    x = np.expand_dims(x, axis=0)\n\n    x = preprocess_input(x)\n\n    preds = model.predict(x)\n    \n    # Extra shift to remove 'background' as entry0\n    preds_1000 = preds[:, 1:]\n    \n    print('Predicted:', decode_predictions(preds_1000))\n"

In [7]:
# import the model
model = InceptionV1(include_top=True, weights=None, input_tensor=None, input_shape=None, pooling=None, classes=36)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
Conv2d_1a_7x7_conv (Conv2D)     (None, 112, 112, 64) 9408        input_1[0][0]                    
__________________________________________________________________________________________________
Conv2d_1a_7x7_bn (BatchNormaliz (None, 112, 112, 64) 192         Conv2d_1a_7x7_conv[0][0]         
__________________________________________________________________________________________________
Conv2d_1a_7x7_act (Activation)  (None, 112, 112, 64) 0           Conv2d_1a_7x7_bn[0][0]           
__________________________________________________________________________________________________
MaxPool_2a

In [8]:
# set up training parameters
WEIGHT_DECAY = 0.0005

loss_metric = "binary_crossentropy" if N_CATEGORY == 2 else "categorical_crossentropy"
model.compile(loss=loss_metric, metrics=["accuracy"], 
              optimizer=optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=WEIGHT_DECAY, amsgrad=False))

routine_dir = model_dir + "routine-{epoch:02d}-{val_acc:.2f}.hdf5"
save_models = ModelCheckpoint(routine_dir, monitor='acc', verbose=0, save_best_only=False, save_weights_only=False, mode='auto', period=10)
best_dir = model_dir + "best-{epoch:02d}-{val_acc:.2f}.hdf5"
save_best = ModelCheckpoint(best_dir, monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=False, mode='auto', period=1)

# train
res = model.fit_generator(
    train_generator, steps_per_epoch = math.ceil(4229/BATCH_SIZE),
    epochs=190, verbose=2, callbacks=[save_models, save_best],
    validation_data=val_generator, validation_steps = math.ceil(846/BATCH_SIZE)
)
#5075/BATCH_SIZE

Epoch 1/190
 - 43s - loss: 1.3059 - acc: 0.5532 - val_loss: 6.8040 - val_acc: 0.1891
Epoch 2/190
 - 19s - loss: 0.5398 - acc: 0.8151 - val_loss: 10.7390 - val_acc: 0.0946
Epoch 3/190
 - 20s - loss: 0.3951 - acc: 0.8749 - val_loss: 2.3217 - val_acc: 0.5343
Epoch 4/190
 - 20s - loss: 0.2991 - acc: 0.9088 - val_loss: 2.2815 - val_acc: 0.4362
Epoch 5/190
 - 19s - loss: 0.3127 - acc: 0.9053 - val_loss: 4.5521 - val_acc: 0.3014
Epoch 6/190
 - 19s - loss: 0.2305 - acc: 0.9315 - val_loss: 8.4917 - val_acc: 0.1277
Epoch 7/190
 - 19s - loss: 0.1670 - acc: 0.9578 - val_loss: 0.4302 - val_acc: 0.8475
Epoch 8/190
 - 20s - loss: 0.1641 - acc: 0.9569 - val_loss: 2.2126 - val_acc: 0.4054
Epoch 9/190
 - 20s - loss: 0.1416 - acc: 0.9630 - val_loss: 1.4626 - val_acc: 0.6017
Epoch 10/190
 - 20s - loss: 0.1092 - acc: 0.9722 - val_loss: 1.5202 - val_acc: 0.6501
Epoch 11/190
 - 20s - loss: 0.1104 - acc: 0.9703 - val_loss: 1.2016 - val_acc: 0.6111
Epoch 12/190
 - 20s - loss: 0.0970 - acc: 0.9729 - val_loss: 0

Epoch 94/190
 - 19s - loss: 0.0429 - acc: 0.9879 - val_loss: 0.1545 - val_acc: 0.9752
Epoch 95/190
 - 20s - loss: 0.0029 - acc: 1.0000 - val_loss: 0.1366 - val_acc: 0.9799
Epoch 96/190
 - 19s - loss: 0.0034 - acc: 0.9998 - val_loss: 0.1285 - val_acc: 0.9811
Epoch 97/190
 - 19s - loss: 0.0017 - acc: 0.9998 - val_loss: 0.1120 - val_acc: 0.9811
Epoch 98/190
 - 20s - loss: 0.0012 - acc: 0.9998 - val_loss: 0.1219 - val_acc: 0.9811
Epoch 99/190
 - 20s - loss: 0.0015 - acc: 0.9995 - val_loss: 0.1312 - val_acc: 0.9787
Epoch 100/190
 - 20s - loss: 7.8875e-04 - acc: 0.9998 - val_loss: 0.1244 - val_acc: 0.9799
Epoch 101/190
 - 20s - loss: 0.0164 - acc: 0.9951 - val_loss: 1.1665 - val_acc: 0.7364
Epoch 102/190
 - 20s - loss: 0.0233 - acc: 0.9931 - val_loss: 0.3466 - val_acc: 0.9078
Epoch 103/190
 - 19s - loss: 0.0311 - acc: 0.9935 - val_loss: 0.6445 - val_acc: 0.8404
Epoch 104/190
 - 20s - loss: 0.0117 - acc: 0.9969 - val_loss: 0.1458 - val_acc: 0.9740
Epoch 105/190
 - 19s - loss: 0.0118 - acc: 0.

Epoch 186/190
 - 19s - loss: 6.6006e-04 - acc: 1.0000 - val_loss: 0.1289 - val_acc: 0.9811
Epoch 187/190
 - 20s - loss: 4.3158e-04 - acc: 1.0000 - val_loss: 0.1278 - val_acc: 0.9823
Epoch 188/190
 - 19s - loss: 7.3192e-04 - acc: 1.0000 - val_loss: 0.1413 - val_acc: 0.9775
Epoch 189/190
 - 19s - loss: 0.0022 - acc: 0.9998 - val_loss: 0.1302 - val_acc: 0.9823
Epoch 190/190
 - 20s - loss: 2.7668e-04 - acc: 1.0000 - val_loss: 0.1319 - val_acc: 0.9823


In [ ]:
'''
scores = model.evaluate(x_train, y_train, verbose=0)
print("Train Accuracy: %.2f%%" % (scores[1]*100))
scores = model.evaluate(x_test, y_test, verbose=0)
print("Test Accuracy: %.2f%%" % (scores[1]*100))
'''